# Load images into table

This demonstrates different ways to load images into a database table.

We use the script called <em>madlib_image_loader.py</em> located at https://github.com/apache/madlib-site/tree/asf-site/community-artifacts/Deep-learning which uses the Python Imaging Library so supports multiple formats http://www.pythonware.com/products/pil/

## Table of contents

<a href="#setup">1. Setup image loader</a>

<a href="#fetch_numpy">2. Fetch images then load NumPy array into table</a>

<a href="#file_system">3. Load from file system into table</a>

<a href="#fetch_numpy_label">4. Fetch images then load NumPy array into table with label datatype as INT</a>

In [1]:
%load_ext sql

In [2]:
# Greenplum Database 5.x on GCP - via tunnel
%sql postgresql://gpadmin@localhost:8000/madlib
        
# PostgreSQL local
#%sql postgresql://fmcquillan@localhost:5432/madlib

In [3]:
%sql select madlib.version();
#%sql select version();

1 rows affected.


version
"MADlib version: 1.18.0-dev, git revision: rel/v1.17.0-89-g14a91ce, cmake configuration time: Fri Mar 5 23:08:38 UTC 2021, build type: release, build system: Linux-3.10.0-1160.11.1.el7.x86_64, C compiler: gcc 4.8.5, C++ compiler: g++ 4.8.5"


<a id="setup"></a>
# 1. Set up image loader

We use the script called <em>madlib_image_loader.py</em> located at https://github.com/apache/madlib-site/tree/asf-site/community-artifacts/Deep-learning

In [4]:
import sys
import os
from keras.datasets import cifar10

madlib_site_dir = '/Users/fmcquillan/Documents/Product/MADlib/Demos/data'
sys.path.append(madlib_site_dir)

# Import image loader module
from madlib_image_loader import ImageLoader, DbCredentials

# Specify database credentials, for connecting to db
#db_creds = DbCredentials(user='gpadmin',
#                         host='35.239.240.26',
#                         port='5432',
#                         password='')

# Specify database credentials, for connecting to db
db_creds = DbCredentials(user='fmcquillan',
                          host='localhost',
                          port='5432',
                          password='')

# Initialize ImageLoader (increase num_workers to run faster)
iloader = ImageLoader(num_workers=5, db_creds=db_creds)

Using TensorFlow backend.


<a id="fetch_numpy"></a>
# 2. Fetch images then load NumPy array into table

<em>iloader.load_dataset_from_np(data_x, data_y, table_name, append=False)</em>

- <em>data_x</em> contains image data in np.array format


- <em>data_y</em> is a 1D np.array of the image categories (labels).


- If the user passes a <em>table_name</em> while creating ImageLoader object, it will be used for all further calls to load_dataset_from_np.  It can be changed by passing it as a parameter during the actual call to load_dataset_from_np, and if so future calls will load to that table name instead.  This avoids needing to pass the table_name again every time, but also allows it to be changed at any time.

In [5]:
# Load dataset into np array
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [6]:
%sql DROP TABLE IF EXISTS cifar_10_train_data, cifar_10_test_data;

# Save images to temporary directories and load into database
iloader.load_dataset_from_np(x_train, y_train, 'cifar_10_train_data', append=False)
iloader.load_dataset_from_np(x_test, y_test, 'cifar_10_test_data', append=False)

Done.


[]

MainProcess: Connected to madlib db.
Executing: CREATE TABLE cifar_10_train_data (id SERIAL, x REAL[], y TEXT)


RuntimeError: relation "cifar_10_train_data" already exists while creating cifar_10_train_data in db madlib.
If the table already exists, you can use append=True to append more images to it.

In [12]:
%%sql
SELECT COUNT(*) FROM cifar_10_train_data;

1 rows affected.


count
50000


In [13]:
%%sql
SELECT COUNT(*) FROM cifar_10_test_data;

1 rows affected.


count
10000


<a id="file_system"></a>
# 3. Load from file system

Uses the Python Imaging Library so supports multiple formats
http://www.pythonware.com/products/pil/

<em>load_dataset_from_disk(root_dir, table_name, num_labels='all', append=False)</em>

- Calling this function  will look in <em>root_dir</em> on the local disk of wherever this is being run.  It will skip over any files in that directory, but will load images contained in each of its subdirectories.  The images should be organized by category/class, where the name of each subdirectory is the label for the images contained within it.


- The <em>table_name</em> and <em>append</em> parameters are the same as above  The parameter <em>num_labels</em> is an optional parameter which can be used to restrict the number of labels (image classes) loaded, even if more are found in <em>root_dir</em>.  For example, for a large dataset you may have hundreds of labels, but only wish to use a subset of that containing a few dozen.

For example, if we put the CIFAR-10 training data is in 10 subdirectories under directory <em>cifar10</em>, with one subdirectory for each class:

In [14]:
%sql drop table if exists cifar_10_train_data_filesystem;
# Load images from file system
iloader.load_dataset_from_disk('/Users/fmcquillan/tmp/cifar10', 'cifar_10_train_data_filesystem', num_labels='all', append=False)

Done.
MainProcess: Connected to madlib db.
Executing: CREATE TABLE cifar_10_train_data_filesystem (id SERIAL, x REAL[], y TEXT,                        img_name TEXT)
CREATE TABLE
Created table cifar_10_train_data_filesystem in madlib db
.DS_Store is not a directory, skipping
number of labels = 10
Found 10 image labels in /Users/fmcquillan/tmp/cifar10
Spawning 5 workers...
Initializing PoolWorker-11 [pid 82438]
PoolWorker-11: Created temporary directory /tmp/madlib_aEC1lF2HqL
Initializing PoolWorker-12 [pid 82439]
PoolWorker-12: Created temporary directory /tmp/madlib_70qpwFzzqW
Initializing PoolWorker-13 [pid 82440]
PoolWorker-13: Created temporary directory /tmp/madlib_r2u4Zo5bPt
PoolWorker-11: Connected to madlib db.
Initializing PoolWorker-14 [pid 82441]
PoolWorker-12: Connected to madlib db.
PoolWorker-14: Created temporary directory /tmp/madlib_aTPESoNjVi
Initializing PoolWorker-15 [pid 82443]
PoolWorker-13: Connected to madlib db.
PoolWorker-15: Created temporary directory /tmp/m

PoolWorker-11: Wrote 1000 images to /tmp/madlib_aEC1lF2HqL/cifar_10_train_data_filesystem0008.tmp
PoolWorker-14: Loaded 1000 images into cifar_10_train_data_filesystem
PoolWorker-15: Loaded 1000 images into cifar_10_train_data_filesystem
PoolWorker-12: Loaded 1000 images into cifar_10_train_data_filesystem
PoolWorker-13: Loaded 1000 images into cifar_10_train_data_filesystem
PoolWorker-11: Loaded 1000 images into cifar_10_train_data_filesystem
PoolWorker-14: Wrote 1000 images to /tmp/madlib_aTPESoNjVi/cifar_10_train_data_filesystem0009.tmp
PoolWorker-15: Wrote 1000 images to /tmp/madlib_rhVwjLTbWI/cifar_10_train_data_filesystem0009.tmp
PoolWorker-12: Wrote 1000 images to /tmp/madlib_70qpwFzzqW/cifar_10_train_data_filesystem0009.tmp
PoolWorker-13: Wrote 1000 images to /tmp/madlib_r2u4Zo5bPt/cifar_10_train_data_filesystem0009.tmp
PoolWorker-11: Wrote 1000 images to /tmp/madlib_aEC1lF2HqL/cifar_10_train_data_filesystem0009.tmp
PoolWorker-14: Loaded 1000 images into cifar_10_train_data_fil

In [15]:
%%sql
SELECT COUNT(*) FROM cifar_10_train_data_filesystem;

1 rows affected.


count
50000


<a id="fetch_numpy_label"></a>
# 4. Fetch images then load NumPy array into table with label datatype as INT

<em>iloader.load_dataset_from_np(data_x, data_y, table_name, append=False, label_datatype= 'int')</em>

- <em>data_x</em> contains image data in np.array format


- <em>data_y</em> is a 1D np.array of the image categories (labels).


- If the user passes a <em>table_name</em> while creating ImageLoader object, it will be used for all further calls to load_dataset_from_np.  It can be changed by passing it as a parameter during the actual call to load_dataset_from_np, and if so future calls will load to that table name instead.  This avoids needing to pass the table_name again every time, but also allows it to be changed at any time.


- If the user passes a <em>label_datatype</em> is the datatype of the image categories (labels) column in the output table.

In [14]:
# Load dataset into np array
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

In [15]:
%sql DROP TABLE IF EXISTS cifar_10_train_data_int, cifar_10_test_data_int;

# Save images to temporary directories and load into database
iloader.load_dataset_from_np(x_train, y_train, 'cifar_10_train_data_int', append=False, label_datatype= 'int')
iloader.load_dataset_from_np(x_test, y_test, 'cifar_10_test_data_int', append=False, label_datatype= 'int')

Done.
MainProcess: Connected to madlib db.
Executing: CREATE TABLE cifar_10_train_data_int (id SERIAL, x REAL[], y int)
CREATE TABLE
Created table cifar_10_train_data_int in madlib db
Spawning 5 workers...
Initializing PoolWorker-11 [pid 42137]
PoolWorker-11: Created temporary directory /tmp/madlib_8Pc3rPCJJL
Initializing PoolWorker-12 [pid 42138]
PoolWorker-12: Created temporary directory /tmp/madlib_QOyefNO7bi
Initializing PoolWorker-13 [pid 42139]
Initializing PoolWorker-14 [pid 42140]
PoolWorker-13: Created temporary directory /tmp/madlib_iFiR9zT8Za
PoolWorker-14: Created temporary directory /tmp/madlib_EAo5JPfg55
Initializing PoolWorker-15 [pid 42142]
PoolWorker-11: Connected to madlib db.
PoolWorker-15: Created temporary directory /tmp/madlib_AjkjHXX3lr
PoolWorker-12: Connected to madlib db.
PoolWorker-13: Connected to madlib db.
PoolWorker-14: Connected to madlib db.
PoolWorker-15: Connected to madlib db.
PoolWorker-11: Wrote 1000 images to /tmp/madlib_8Pc3rPCJJL/cifar_10_train_

In [16]:
%%sql
SELECT COUNT(*) FROM cifar_10_train_data_int;

1 rows affected.


count
50000


In [17]:
%%sql
SELECT COUNT(*) FROM cifar_10_test_data_int;

1 rows affected.


count
10000
